<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/callbacks/OpenInferenceCallback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UpTrain Callback Handler



## Install Dependencies and Import Libraries

Install notebook dependencies.

In [ ]:
!pip install -q html2text llama-index pandas tqdm uptrain cohere

Import libraries.


In [ ]:
from llama_index import (
    ServiceContext,
    VectorStoreIndex,
)
from llama_index.node_parser import SentenceSplitter
from llama_index.readers import SimpleWebPageReader
from llama_index.callbacks import CallbackManager, UpTrainCallbackHandler
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.node_parser import SentenceSplitter
from llama_index.service_context import set_global_service_context
from llama_index.query_engine.sub_question_query_engine import (
    SubQuestionQueryEngine,
)
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.tools.types import ToolMetadata
from llama_index.llms import OpenAI
from llama_index.query_engine import FLAREInstructQueryEngine
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.callbacks import CallbackManager, UpTrainCallbackHandler

## Setup

Instantiate the UpTrain callback handler and attach to the service context.

In [ ]:
callback_handler = UpTrainCallbackHandler(
    uptrain_api_key="up-************************"
)
callback_manager = CallbackManager([callback_handler])
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager
)
set_global_service_context(service_context)

## Load and Parse Documents

Load documents from Paul Graham's essay "What I Worked On".

In [ ]:
documents = SimpleWebPageReader().load_data(
    [
        "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt"
    ]
)
# print(documents[0].text)

Parse the document into nodes. Display the first node's text.

In [ ]:
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
print(nodes[0].text)

# 1. Query

Build and run your query engine across a collection of queries.

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()

max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)

# 2. Sub-Question

### Setup sub question query engine

In [ ]:
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents=documents, use_async=True, service_context=service_context
).as_query_engine()

query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="documents",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

response = query_engine.query(
    "How was Paul Grahams life different before, during, and after YC?"
)

# 3. Re-ranking

In [ ]:
# Recipe
# Define a Postprocessor object, here CohereRerank
# Build QueryEngine that uses this Postprocessor on retrieved docs

# Build CohereRerank post retrieval processor
api_key = "**********************************"  # insert cohere API key here
cohere_rerank = CohereRerank(api_key=api_key, top_n=5)

index = VectorStoreIndex.from_documents(
    documents=documents, service_context=service_context
)
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
    service_context=service_context,
)

# Use your advanced RAG
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

# 4. Generator-Enhanced Retrieval

In [ ]:
### Recipe
### Build a FLAREInstructQueryEngine which has the generator LLM play
### a more active role in retrieval by prompting it to elicit retrieval
### instructions on what it needs to answer the user query.

callback_handler = UpTrainCallbackHandler(
    uptrain_api_key="up-*************************"
)
callback_manager = CallbackManager([callback_handler])
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager, llm=OpenAI(model="gpt-4")
)

# Build FLAREInstructQueryEngine
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
index_query_engine = index.as_query_engine(
    similarity_top_k=2, service_context=service_context
)
flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    service_context=service_context,
    max_iterations=7,
    verbose=True,
)

# Use your advanced RAG
response = flare_query_engine.query(
    "Can you tell me about the author's trajectory in the startup world?"
)